# Tuning Script

In [ ]:
# Parameters subject to change in every script - involved in naming

MODEL = 'transformer'
FEATURE_IMPORTANCE_ORDERING = 'xgb'
PREPROCESSING = 'Baseline'
TUNER = 'jiaochengb'
FEATURE_ENGINEERING = 'bow'

---

In [ ]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

In [ ]:
import sys
import os
import pickle
from scipy.sparse import load_npz

if COLAB_ENVIRONMENT:
    py_file_location = "./drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/PrivatePackages" # my private packages are stored here
    home_directory = './drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/' # my home directory is stored in ./LAB of google drive
    if MODEL == 'ebc':
        !pip install interpret==0.5.0
    elif MODEL == 'cbc':
        !pip install catboost
else:
    py_file_location = '../../PrivatePackages'
    home_directory = '../../'

sys.path.append(os.path.abspath(py_file_location))

from Tuners import JiaoChengB

import pandas as pd

In [ ]:
## Parameters that are same for every script

SEED = 2608

TASK_TYPE = 'Classification' # this task is a classification task

TUNE_FEATURES_AS_HYPERPARAMETERS = True # we have decided to tune features as hp

---

In [ ]:
# model to use to tune
from model_class.Transformer import Transformer as clf

# what values to try for each hyperparameter
parameter_choices = {
    'hidden_dim': (16, 32, 64, 128),
    'num_transformer_layers': (1, 2, 3),
    'dropout': (0, 0.05, 0.1, 0.15, 0.2),
    'dim_feedforward': (32, 64, 128, 256, 512),
    'batch_size': (16, 32, 64, 128, 256),
    'nhead': (2, 4, 8, 16),
    'use_cls': (False, True),
    'epochs': (5, 10, 15, 20),
    'lr': (1e-4, 1e-3, 1e-2, 1e-1),
    'batchnorm': (False, True),
    'grad_clip': (False, True),
}

# what values to set non-tuneable parameters/hyperparameters
non_tunable_hyperparameters_dict = {
                                    'random_state': SEED,
                                    'loss': nn.CrossEntropyLoss(),
                                    'rootpath': './',
                                    'TabularDataFactory': TabularDataFactory,
                                    'TabularDataset': TabularDataset,
                                    'name': 'MLP',
                                    'output_dim': 2,
                                    'mode': 'Classification',
                                    }

# what order to tune hyperparameters in (JiaoCheng specific)
tuning_order = ['features',
                'num_transformer_layers',
                'nhead',
                'hidden_dim',
                'dim_feedforward',
                'dropout',
                'use_cls',
                'batchnorm',
                'grad_clip',
                'lr',
                'epochs',
                'batch_size'
                ]

# what order to tune hyperparameters in (JiaoCheng specific)
default_hyperparameter_values = {
                                'features': 0,
                                'num_transformer_layers': 1,
                                'nhead': 4,
                                'hidden_dim': 16,
                                'dim_feedforward': 32,
                                'dropout': 0.1,
                                'use_cls': True,
                                'batchnorm': False,
                                'grad_clip': True,
                                'lr': 1e-2,
                                'epochs': 10,
                                'batch_size': 32
                                }

---

Read in and Prepare Data

In [ ]:
train_x = load_npz(home_directory + f'data/curated/baseline/train_{FEATURE_ENGINEERING}_x.npz')
val_x = load_npz(home_directory + f'data/curated/baseline/val_{FEATURE_ENGINEERING}_x.npz')
test_x = load_npz(home_directory + f'data/curated/baseline/test_{FEATURE_ENGINEERING}_x.npz')

with open(home_directory + 'data/curated/baseline/train_y.pkl', 'rb') as f:
    train_y = pickle.load(f)
with open(home_directory + 'data/curated/baseline/val_y.pkl', 'rb') as f:
    val_y = pickle.load(f)
with open(home_directory + 'data/curated/baseline/test_y.pkl', 'rb') as f:
    test_y = pickle.load(f)

In [ ]:
# Import feature importance ordering

with open(f'{home_directory}models/{FEATURE_IMPORTANCE_ORDERING}_{FEATURE_ENGINEERING}_feature_importance_ordering_{PREPROCESSING}.pickle', 'rb') as f:
    feature_importance_ordering = pickle.load(f)

Set parameters

In [ ]:
# initialisation
tuner = JiaoChengB.JiaoChengB()

print('---')

# define what model we are tuning
tuner.read_in_model(clf, TASK_TYPE)

print('---')

# read in the data for training and validation
tuner.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

print('---')

# set what hp values to tune
tuner.set_hyperparameters(parameter_choices)
# WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up hp values that need to be changed from default but NOT to be tuned
tuner.set_non_tuneable_hyperparameters(non_tunable_hyperparameters_dict)

print('---')

# set up feature importance ordering

if TUNE_FEATURES_AS_HYPERPARAMETERS:
  tuner.set_features(feature_importance_ordering)
  # WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up the order of hyperparameters when iteratively tuning using JiaoCheng
tuner.set_tuning_order(tuning_order)

print('---')

# set up the default hp values for first iteration of tuning JiaoCheng
tuner.set_hyperparameter_default_values(default_hyperparameter_values)

print('---')

try: # try to read in previous results to continue tuning ...
  tuner.read_in_tuning_result_df(f'{home_directory}models/tuning/{TUNER}_{MODEL}_{FEATURE_IMPORTANCE_ORDERING}_{FEATURE_ENGINEERING}_{PREPROCESSING}.csv')
except:
  print('No previous tuning data read in')

print('---')

# set up where to save the tuning result csv
tuner.set_tuning_result_saving_address(f'{home_directory}models/tuning/{TUNER}_{MODEL}_{FEATURE_IMPORTANCE_ORDERING}_{FEATURE_ENGINEERING}_{PREPROCESSING}.csv')

print('---')

# set up where to save the current best model
# tuner.set_best_model_saving_address(f'{home_directory}models/tmp_models/{TUNER}_{MODEL}_{FEATURE_IMPORTANCE_ORDERING}_{FEATURE_ENGINEERING}_{PREPROCESSING}')

In [ ]:
# ignore warnings

import warnings
warnings.filterwarnings("ignore")

In [10]:
# begin tuning ... (or continue training)

tuner.tune()

Streaming output truncated to the last 5000 lines.
Trained and Tested combination 405 of 593250: (3, 5, 0, 2, 4, 76), taking 78.36 seconds to get val score of 0.7778
        Current best combo: (3, 5, 0, 2, 2, 76) with val score 0.8407

Best combo after this hyperparameter: (3, 5, 0, 2, 2, 76) , UPDATED SINCE LAST HYPERPARAMETER


FEATURE_INDEX: 0 (n_estimators) ROUND 4

Round 4 
Hyperparameter: features (index: 5) 

Trained and Tested combination 406 of 593250: (3, 5, 0, 2, 2, 0), taking 1.09 seconds to get val score of 0.7885
        Current best combo: (3, 5, 0, 2, 2, 76) with val score 0.8407
Trained and Tested combination 407 of 593250: (3, 5, 0, 2, 2, 1), taking 1.57 seconds to get val score of 0.7856
        Current best combo: (3, 5, 0, 2, 2, 76) with val score 0.8407
Trained and Tested combination 408 of 593250: (3, 5, 0, 2, 2, 2), taking 1.84 seconds to get val score of 0.7826
        Current best combo: (3, 5, 0, 2, 2, 76) with val score 0.8407
Trained and Tested combination